In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import seaborn as sns

genome_ids_file="../../pangenome-repo/Pangenome-Analysis-Workflow/genome_ids/Campylobacter_coli_genome_ids.csv"
clstr_freq_file="../../pangenome-repo/Pangenome-Analysis-Workflow/codes/Campylobacter_coli/Campylobacter_coli_cluster_frequencies.csv"
clstr_file="../../pangenome-repo/Pangenome-Analysis-Workflow/codes/Campylobacter_coli/Campylobacter_coli.fasta.clstr"
clstr_fasta_file="../../pangenome-repo/Pangenome-Analysis-Workflow/codes/Campylobacter_coli/Campylobacter_coli.fasta"

# Creating presence-absence matrix

In [2]:
samples_df = pd.read_csv(genome_ids_file, dtype=str)
samples_list=samples_df['genome.genome_ids'].tolist()

clstr_freq_df=pd.read_csv(clstr_freq_file, index_col=1)
clstr_freq_df=clstr_freq_df.drop(clstr_freq_df.columns[0], axis=1)
clstr_list=clstr_freq_df.index.tolist()

df = pd.DataFrame(index=samples_list, columns=clstr_list)
df = df.fillna(0)

In [5]:
with open(clstr_file) as f:
    for line in f:
        if line.startswith(">Cluster"):
            #if its a > line get the cluster id and start counting its occurences in which samples by saving it to the local var
            cluster_id=line[1:].strip()
        else:
            #else its a line designating the sample genome, one that the last cluster_id is present in
            # get the 1st group matching here \d\t\d+aa, >fig\|([\d\.\d]+).+

            genome_id=re.match(r"\d+\t\d+aa, >fig\|([\d\.\d]+).+", line).group(1)
            genome_id=genome_id[:-1]  #removing the trailing .
            df.loc[genome_id, cluster_id]+=1

In [7]:
df.to_csv("../../data/presence_matrices/Campylobacter_coli_presence_absence.csv")

## Save cluster names 

In [12]:
dict={}
with open(clstr_fasta_file) as f:
    for line in f:
        if line.startswith(">"):
            pattern=">fig\|(\d+\.\d+).peg.+? (.+)"
            id=re.match(pattern, line).group(1)
            desc=re.match(pattern, line).group(2)
            dict[id]=desc

pd.DataFrame.from_dict(dict, orient='index')

<>:5: SyntaxWarning: invalid escape sequence '\|'
<>:5: SyntaxWarning: invalid escape sequence '\|'
/tmp/ipykernel_11299/3219330680.py:5: SyntaxWarning: invalid escape sequence '\|'
  pattern=">fig\|(\d+\.\d+).peg.+? (.+)"


,0
195.2024,Cysteine desulfurase (EC 2.8.1.7)
195.2025,hypothetical protein
195.2026,FIG00470572: hypothetical protein
195.2027,hypothetical protein
195.2028,hypothetical protein
...,...
195.2313,hypothetical protein
195.2314,Putative transferase
195.2315,"Pseudaminic acid biosynthesis protein PseA, po..."
195.2316,hypothetical protein
